In [ ]:
import pickle
import numpy as np

In [ ]:
xgb_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/with_feature_aggregration/testing/test_gt_and_pred.pkl'
transformer_predictions_path = '/Users/jk1/Downloads/transformer_20230402_184459_test_set_evaluation/fold_2_test_gt_and_pred.pkl'

In [ ]:
xgb_gt, xgb_predictions = pickle.load(open(xgb_predictions_path, 'rb'))
transformer_gt, transformer_predictions = pickle.load(open(transformer_predictions_path, 'rb'))

In [ ]:
all(xgb_gt == transformer_gt)

Using DeLong's test to compare the two models

In [ ]:
from prediction.outcome_prediction.model_comparison.de_long_comparison import delong_roc_test

10 ** delong_roc_test(xgb_gt, xgb_predictions, transformer_predictions)

Using the pyroc package to compare the two models

In [ ]:
import pyroc

roc = pyroc.ROC(xgb_gt, [xgb_predictions, transformer_predictions])

In [ ]:
roc.compare(contrast=[1, -1], alpha=0.05)